In [1]:
# Many thanks to Theresa, Elana, Ariana, and Sarah T for working on this with me and helping me - Some of this code is theirs
# Much of this code is from lectures 8 and 9 

In [ ]:
# THIS IS FOR QUESTION 1

In [2]:
#Import the packages needed to do the hw
import re

import Bio.SeqIO

In [3]:
#read in the file with all of the seq reads and all of the other data - make sure Python knows it is a fastq file
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

In [4]:
#count how many reads are in the file
print(f"Found {len(seqreads)} sequencing reads.")

Found 10000 sequencing reads.


In [5]:
# Makes a list of just the sequencing reads from the file - excludes other data
seqreads_str = [] 
for seqrecord in seqreads:
    sequence = str(seqrecord.seq) # convert sequence part to string
    seqreads_str.append(sequence) # add string sequence to list
    
#print(seqreads_str)
#Just checking :)

In [6]:
#Doube check that there are 10000 seqs in the seq list made in previous step
assert len(seqreads_str) == len(seqreads)

In [7]:
 #what is the sequence of the first item in the list (just checking) and also so I can compare to a reverse seq later
seqreads_str[0]

'GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA'

In [8]:
#Defining a function that will return the reverse complement of an inputted sequence
def reverse_complement(seq, unk_partner = 'N'):
    """Get reverse complement of a DNA sequence"""
    
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    rseq = ''
    for a in seq:
        if a in base_partner:
            # look up the complementary base in tATGGTGCATCATTATATGCAAATCCGGhe dictionary
            pair = base_partner[a]
            rseq = pair + rseq
        else:
            rseq = unk_partner + rseq
    return rseq

In [9]:
#Making a new list that is all of the reverse complements of the seqs from seqreads_str
rev = []

for seq in seqreads_str:
    reverse_complement(seq)
    rev.append(reverse_complement(seq))

In [10]:
#Double checking that it's still a list
type(rev)

list

In [11]:
#Double checking that the list still has 10000 seqs 
len(rev)

10000

In [12]:
#Looking at the first 5 seqs in the new list - looks good!
rev[:5]

['TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTTTAGTGCTGCGTTTACAAAACTGGTTTCAAAATTATTAATTTCAGCTTTTTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTTGGCCTGATGGAGCAGAACTCCCTTTCACGATAGATAAATAATAGTGCACCATAGGCGGCCGCACTAAATAACTTAAGC',
 'TCGTACACTCTGTCATTAGGGATGTATTTGTTTAATGCATGGGGTTGTATACTAAGCTGTTGATGGTGTCTTTATGATACCTATTTAGGTTTTTTTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTATGTGTTCCAACGGCTCCTTGCAATGCCGGATTTGCATATAATGATGCACCATAGGCGGCCGCAACCGTGACCAGGAAG',
 'TCGTAGTGTATAGTAGAAGGGACGTCTACGTTAATCAGTGTCATAAGTTCGATCATTGGCTTCAATACCTAGTGGGAGTTAGATTTTGTATGTTAGTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTTGGCCTGATGGAGCAGAACTCCCTTTCACGATAGATAAATAATAGTGCACCATAGGCGGCCGCGGGGTAATAGCCGTG',
 'TCAGATGAATGGTAGTTGGTGATAGCATGAGGTTGGGTCGGATGGTTAGTGTCTTCTTTCATCTGATGGTTGTGTTTTGCGTGCTTTGCTTTTTTTTTTTTTTTTCAAGCAGAAGACGGCATACGAGATTCTGACATGTGACTGGAGTTCAGACGTGTGCTCTTCCGATCTATGTGTTCCAACGGCTCCTTGCAATGCCGGATTTGCATATAATGATGCAACATAGGCGGCCG

In [13]:
#defining the function count_prot. The arguements are a srting (called seqs), and the parameters are protlen, bclen, and downstream
def count_prot(seqs, protlen=27, bclen=16, downstream='AGGCGGCCGC'):  
    """Count the number of reads with a specific protein sequence upstream of a barcode
      
      Parameters
    ----------
    seqs : str
        Nucleotide sequence matching
    protlen : int
        Length of HA or NA end
    bclen : int
        Length of barcode
    downstream: str
        Sequence downstream of the barcode
        
    """
    
    
    # compiling the reguar expression to run within the function count_prot
    # We want sequence that starts with a a string of any ATCGs that is 27bp long, followed by the downstream constant sequence, followed by any barcode 
    #that is made of ATCGNs and has a length of 16 right at the end of the sequence
    prot_re = re.compile("(?P<prot>[ATCG]{" + str(protlen) + "})"+ downstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    #Making objects specific to the function count_prot
    NA = 'CACGATAGATAAATAATAGTGCACCAT'
    HA = 'CCGGATTTGCATATAATGATGCACCAT'
    
    #defining a dictionary for the output of the fucntion count_prot 
    prot_counts = {'NA_counts':0, 'HA_counts':0, 'neither':0} 
    
    #Making a for loop to use the regex prot_re where seq is anything and seqs is the arguement in the defined function
    for seq in seqs:
        match = prot_re.search(seq)
        
        #Looks to see if each sequence is a match. If so, group it by the part defined at Prot in the regex
        #if that prot string is the same as the NA object, add 1 count to the NA_counts in the dictionary. Same for HA. 
        #if the prot doesn't match the HA or NA object, add 1 count to neither in the dictionary
        if match: 
            prot = match.group("prot")
            if prot == NA:
                prot_counts["NA_counts"] +=1
            elif prot == HA:
                prot_counts["HA_counts"] += 1
            else:
                prot_counts["neither"] += 1
                
    #When the function is run, spit out the resulting prot_counts dictionary
    return prot_counts

In [ ]:
# BELOW IS THE ANSWER TO NUMBER 1

In [14]:
#Run the count_prot function on the list rev (which is a list of all of the reverse complements of the seqs in seqreads)
count_prot(rev)

{'NA_counts': 3910, 'HA_counts': 5246, 'neither': 248}

In [ ]:
# THIS IS THE CODE FOR QUESTION NUMBER 2

In [18]:
#Define a function that "Identify barcode with known upstream sequence"
def read_barcode(seqs, bclen=16, downstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
    """
    # compiling the reguar expression to run within the function read_barcode
    # We want sequence that starts with a a string of any ATCGs that is 27bp long, followed by the downstream constant sequence, followed by any barcode 
    #that is made of ATCGNs and has a length of 16 right at the end of the sequence
    barcode_re = re.compile("(?P<prot>[ATCG]{27})" + downstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})")
    
    #find the sequences that match the re
    match = barcode_re.search(seq)
    
    #return the sequences that match the patterm as "barcode, prot"
    if match:
        barcode = match.group('barcode')
        prot = match.group('prot')
        return barcode, prot
    else:
        barcode = "not_match"
        gene = "not_match"
        return barcode, prot

In [25]:
#define the HA and NA specific sequences
NA = 'CACGATAGATAAATAATAGTGCACCAT'
HA = 'CCGGATTTGCATATAATGATGCACCAT'

#Set up empty barcode_count dictionaries
barcode_counts_HA = {}
barcode_counts_NA = {}

#make a for loop to read each sequece in the list and 
#1. decide if it matches the barcode regex and return the matches as "barcode, prot" 
#2 assign the sequence as belonging to HA or NA 
#3 count them up in a dictionary
for seq in rev:
    barcode, prot = read_barcode(seq, bclen = 16)

    if prot == NA:
        if barcode in barcode_counts_NA:
            barcode_counts_NA[barcode] += 1
        else:
            barcode_counts_NA[barcode] = 1
    elif prot == HA:
        if barcode in barcode_counts_HA:
            barcode_counts_HA[barcode] += 1
        else:
            barcode_counts_HA[barcode] = 1


In [ ]:
# BELOW IS THE ANSWER FOR QUESTION NUMBER 2

In [26]:
#Find the barcode with the most counts for each prot (NA or HA) and print that barcode and the count 
max_NA_barcode = max(barcode_counts_NA)
max_NA_count = max(barcode_counts_NA.values())

max_HA_barcode = max(barcode_counts_HA)
max_HA_count = max(barcode_counts_HA.values())


print("the NA barcode with the most counts is", max_NA_barcode, "with", max_NA_count, "counts")
print("the HA barcode with the most counts is", max_HA_barcode, "with", max_HA_count, "counts")

the NA barcode with the most counts is TTTCTCCCGCATCTCG with 152 counts
the HA barcode with the most counts is TTTGTCCCTTGAACCC with 155 counts
